THOMAS MCDONNELL QUANT TRADING STRATEGIES HOMEWORK 3

In [96]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm


import warnings

# Suppress the warning
warnings.filterwarnings("ignore")

In [69]:
# FC = pd.read_csv('/Users/tmac/Downloads/ZACKS_FC_2_76e4bece47ce87cb8f221f639c7f829b.csv')
MT = pd.read_csv('/Users/tmac/Downloads/ZACKS_MT_2_5c2afb6368dcc3ed48e1a84279323e63.csv')
FR = pd.read_csv('/Users/tmac/Downloads/ZACKS_FR_2_f40c6a304f87d9f492c1f21839d474e2.csv')
MKTV = pd.read_csv('/Users/tmac/Downloads/ZACKS_MKTV_2_ecb7f768974bbdd26964caefe2fd0378.zip')



In [102]:
unique_tickers = FR['ticker'].unique().tolist()
x = dict()
for ticker in tqdm(unique_tickers):
    df = FR[FR['ticker'] == ticker]
    x[ticker] = df

100%|██████████| 16146/16146 [08:22<00:00, 32.16it/s]


In [113]:
l = []
for key in x:
    if len(x[key]) > 80:
        l.append(key)


2717


In [71]:
# eod_price = dict()
# for ticker in tick_list:
#     df =  quandl.get_table('WIKI/PRICES', ticker = [ticker], 
#                     qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
#                     date = { 'gte': '2015-12-31', 'lte': '2016-12-31' }, 
#                      paginate=True)
#     eod_price[ticker]  = df

